In [1]:
!pip install fastai
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 78kB/s  eta 0:00:01███▋                  | 244.7MB 26.3MB/s eta 0:00:13 21.9MB/s eta 0:00:16███▉                  | 248.2MB 29.4MB/s eta 0:00:12    44% |██████████████▏                 | 254.9MB 26.1MB/s eta 0:00:13    60% |███████████████████▎            | 346.3MB 71.3MB/s eta 0:00:04    75% |████████████████████████▏       | 434.4MB 74.4MB/s eta 0:00:02    83% |██████████████████████████▉     | 482.9MB 72.1MB/s eta 0:00:02    88% |████████████████████████████▍   | 511.3MB 12.3MB/s eta 0:00:06    88% |████████████████████████████▌   | 512.6MB 38.7MB/s eta 0:00:02    90% |████████████████████████████▉   | 518.6MB 66.6MB/s eta 0:00:01    91% |█████████████████████████████▍  | 528.7MB 62.2MB/s eta 0:00:01


In [2]:
import numpy as np 
import pandas as pd 
from fastai import *
from fastai.text import *

In [3]:
quote = pd.read_json('../input/quotes.json')

In [4]:
print(quote.head)
print(quote.keys())

<bound method NDFrame.head of                                                   Author                        ...                                                                       Tags
0                                              Dr. Seuss                        ...                          [attributed-no-source, cry, crying, experience...
1                                              Dr. Seuss                        ...                          [attributed-no-source, cry, crying, experience...
2                                         Marilyn Monroe                        ...                          [attributed-no-source, best, life, love, mista...
3                                         Marilyn Monroe                        ...                          [attributed-no-source, best, life, love, mista...
4                                         Marilyn Monroe                        ...                          [attributed-no-source, best, life, love, mista...
5               

In [5]:
print(quote.shape)

(48391, 5)


In [ ]:
#quote = quote.drop(quote.columns[[0,3,4,5]], axis=1)

In [6]:
quote['Quote'] = quote['Quote'].str.lower()
print(quote.head())

           Author                        ...                                                                       Tags
0       Dr. Seuss                        ...                          [attributed-no-source, cry, crying, experience...
1       Dr. Seuss                        ...                          [attributed-no-source, cry, crying, experience...
2  Marilyn Monroe                        ...                          [attributed-no-source, best, life, love, mista...
3  Marilyn Monroe                        ...                          [attributed-no-source, best, life, love, mista...
4  Marilyn Monroe                        ...                          [attributed-no-source, best, life, love, mista...

[5 rows x 5 columns]


In [7]:
from pandas import DataFrame
quote = DataFrame.drop_duplicates(DataFrame(quote, columns= ['Quote']))
quote.head()

,Quote
0,"don't cry because it's over, smile because it ..."
2,"i'm selfish, impatient and a little insecure. ..."
5,be yourself; everyone else is already taken.
6,two things are infinite: the universe and huma...
9,"be who you are and say what you feel, because ..."


In [9]:
test_per = 0.05        # test percentage
quote = quote.iloc[np.random.permutation(len(quote))]
piece = int(test_per * len(quote)) + 1
train_quote, test_quote = quote[piece:], quote[:piece]

In [10]:
print(len(train_quote),len(test_quote))

35087 1847


In [11]:
# Pre-processing
nan_row = quote[quote['Quote'].isnull()]
print(nan_row)

Empty DataFrame
Columns: [Quote]
Index: []


In [12]:
data_lm = TextLMDataBunch.from_df('content', train_quote, test_quote, text_cols = 'Quote', label_cols = 'Authors')
#data_clas = TextClasDataBunch.from_df('content', train_quote, test_quote, vocab = data_lm.train_ds.vocab, bs = 32)
#data_lm.save('data_lm_export.pkl')
#data_clas.save('data_clas_export.pkl')

In [13]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult = 0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,4.637690,4.244208,0.257074,00:31


In [14]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
1,4.300571,4.116002,0.270587,00:37


In [20]:
learn.fit(15, lr=1e-3, wd=1e-7)

epoch,train_loss,valid_loss,accuracy,time
1,3.150095,4.138961,0.278949,00:37
2,3.087280,4.161794,0.277043,00:37
3,3.017179,4.192080,0.276631,00:37
4,2.935602,4.222106,0.275086,00:37
5,2.871896,4.251040,0.273214,00:37
6,2.804581,4.288005,0.271806,00:37
7,2.726543,4.315042,0.272184,00:37
8,2.664259,4.348581,0.268939,00:37
9,2.610691,4.370311,0.268098,00:37
10,2.559787,4.393054,0.267256,00:37


In [26]:
learn.predict("xxbos ", n_words=50, temperature=0.75)

'xxbos  i was convinced that all humans were born by god , and that all men are born of god . xxbos your heart perceives which its purpose is ; and how you love it , and how you feel , that is the small way it takes to make your'

In [27]:
number_of_quotes = 500
quotes_counter = 0
all_quotes = []

for i in range(1000):
    quote = learn.predict("xxbos ", n_words=25, temperature=0.8)
    quotes = quote.split("xxbos ")
    quotes = quotes[1:-1]
    
    for quote in quotes:
        quote = quote.replace("xxbos ","").strip()
        if(quote):
            all_quotes.append(quote)
            quotes_counter = quotes_counter+1
            
    if quotes_counter > number_of_quotes:
        break

In [28]:
all_quotes

['i make coffee and make a big cheese in a sandwich . i also use two sorts of potato chips .',
 "the hero is in the tragic moment , and the nobler in one 's own sense of equality .",
 'you shall be able to find peace if you do not seek out the treasures to be loved .',
 "i hope i 'm a success , because that 's the biggest thing i can do .",
 'love is the only weapon in the world , and i will keep myself in fire .',
 "it was a time when a man relates to his own life , one 'd only heaven , a thousand years .",
 'what happens when you see something around you is a moment before you can start .',
 'ignorance is just a way of running faster than walking on the train .',
 "you 're just a little bit annoying and you 're just going to get bored .",
 'life is a gift - without giving it is a gift .',
 'it is we who have told ourselves that god has loved us , and i have discovered that all the time .',
 "do n't forget to treat yourself like a mouse .",
 "if i had a million dollars in love , i wou

In [34]:
outF = open("quote.txt", "w")
for quote_a in all_quotes:
  # write line to output file
  outF.write(quote_a)
  outF.write("\n***************************************************************************************************\n")
outF.close()

In [30]:
learn.predict("xxbos xxfld 1", n_words=35, temperature=0.8)

'xxbos xxfld 1 / 2 [ 10w ] " love brings me thumbs in my pocket . ( 1 / 2 has a very common common sense . mine has more for a lasting relationship than a land'

In [31]:
number_of_quotes1 = 2000
quotes_counter1 = 0
all_quotes1 = []

for i in range(1000):
    quote1 = learn.predict("xxbos ", n_words=25, temperature=0.8)
    quotes1 = quote1.split("xxbos ")
    quotes1 = quotes1[1:-1]
    
    for quote1 in quotes1:
      if ("copyright" or "published") not in quote1:
        quote1 = quote1.replace("xxbos ","").strip()
        if(quote1):
            all_quotes1.append(quote1)
            quotes_counter1 = quotes_counter1+1
            
    if quotes_counter1 > number_of_quotes1:
        break

In [32]:
all_quotes1

["do n't waste your time with somebody who is not afraid of nothing .",
 'the most visible factor is to feel proportion to the abilities , and to improve how you see it .',
 'you know , the truth is , sexy is never going to be a damn damn smart life .',
 'if a door closes , we are let open and pulled into the doorway .',
 "money makes a man buy , but not money . i do n't buy book anonymous .",
 'there is no focus on an ineffable head .',
 'the purpose of life is to be on the side of the majority .',
 'once you learn to read , you learn to judge by the honesty of your right to live .',
 'a desire for self - esteem comes from within . if it is not eternal , then it is eternal .',
 'the language of love is not words but to tears .',
 "i 've got some huge new discoveries across the world . but find them . fantasy is not all there .",
 'to find the slightest balance between genius and stupidity , success is as inevitable as a loss .',
 "do n't do what you want to do , never do it . just go 

In [35]:
outF = open("quote_more.txt", "w")
for quote_a1 in all_quotes1:
  # write line to output file
  outF.write(quote_a1)
  outF.write("\n*******************************************************************************************\n")
outF.close()